<h2> Regression Models with Keras -Artificial Neural Network Algorithm</h2>

<b> By Michael Kumakech<b>

<h2> Introduction </h2> 

Keras is a high-level API for building deep learning models. It has gained favor for its ease of use and syntactic simplicity facilitating fast development. As you will see in this lab and the other labs in this course, building a very complex deep learning network can be achieved with Keras with only few lines of code. You will appreciate Keras even more, once you learn how to build deep models using PyTorch and TensorFlow in the other courses.

<b>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</b>

1. Cement

2. Blast Furnace Slag

3. Fly Ash

4. Water

5. Superplasticizer

6. Coarse Aggregate

7. Fine Aggregate

<b>Let's download the data and read it into a pandas dataframe.</b>

<b> Download the data set </b>

In [1]:
!wget -O  Concrete.csv https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
    

--2022-03-05 13:46:49--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘Concrete.csv’

Concrete.csv        100%[===================>]  57.61K  --.-KB/s    in 0.1s    

2022-03-05 13:46:49 (388 KB/s) - ‘Concrete.csv’ saved [58988/58988]



In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv("Concrete.csv")
df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa.

In [5]:
df.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

<b> Let's check the dataset for any missing values</b>

In [6]:
df.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [7]:
df.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

<b>Split data into predictors and target</b>

The target variable in this problem is the<b> concrete sample strength</b>. Therefore, our predictors will be all the other columns.

In [10]:
df_columns = df.columns

predictors = df[df_columns[df_columns != 'Strength']] # all columns except Strength
target = df['Strength'] # Strength column

<b> Let's do a quick sanity check of the predictors and the target dataframes</b>

In [11]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [12]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

<b> Finally, the last step is to normalize the data by substracting the mean and dividing by the standard deviation.</b>

In [13]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


<b> Let's save the number of predictors to n_cols since we will need this number when building our network.</b>

In [14]:
n_cols = predictors_norm.shape[1] # number of predictors

<h2> Import Keras</h2>

Recall from the videos that Keras normally runs on top of a low-level library such as TensorFlow. This means that to be able to use the Keras library, you will have to install TensorFlow first and when you import the Keras library, it will be explicitly displayed what backend was used to install the Keras library. In CC Labs, we used TensorFlow as the backend to install Keras, so it should clearly print that when we import Keras.

<b> Let's go ahead and import the Keras library</b>

In [15]:
import keras

As you can see, the TensorFlow backend was used to install the Keras library.

Let's import the rest of the packages from the Keras library that we will need to build our regressoin model.

In [16]:
from keras.models import Sequential
from keras.layers import Dense

<h2> Build a Neural Network</h2>

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [18]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(50, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

<b> The above function create a model that has two hidden layers, each of 50 hidden units.</b>

<h2> Train and Test the Network</h2>

<b> Let's call the function now to create our model.</b>

In [19]:
# build the model
model = regression_model()

Next, we will train and test the model at the same time using the fit method. We will leave out 30% of the data for validation and we will train the model for 100 epochs

In [20]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - loss: 1611.7968 - val_loss: 1091.7229 - 531ms/epoch - 23ms/step
Epoch 2/100
23/23 - 0s - loss: 1450.5620 - val_loss: 928.5688 - 97ms/epoch - 4ms/step
Epoch 3/100
23/23 - 0s - loss: 1184.6100 - val_loss: 695.3577 - 139ms/epoch - 6ms/step
Epoch 4/100
23/23 - 0s - loss: 815.3434 - val_loss: 435.0692 - 98ms/epoch - 4ms/step
Epoch 5/100
23/23 - 0s - loss: 466.4427 - val_loss: 242.5246 - 85ms/epoch - 4ms/step
Epoch 6/100
23/23 - 0s - loss: 276.2753 - val_loss: 183.6088 - 122ms/epoch - 5ms/step
Epoch 7/100
23/23 - 0s - loss: 233.1943 - val_loss: 174.6727 - 232ms/epoch - 10ms/step
Epoch 8/100
23/23 - 0s - loss: 218.9008 - val_loss: 171.2889 - 264ms/epoch - 11ms/step
Epoch 9/100
23/23 - 0s - loss: 207.2715 - val_loss: 170.5085 - 145ms/epoch - 6ms/step
Epoch 10/100
23/23 - 0s - loss: 199.2023 - val_loss: 167.8031 - 103ms/epoch - 4ms/step
Epoch 11/100
23/23 - 0s - loss: 191.0759 - val_loss: 167.7532 - 94ms/epoch - 4ms/step
Epoch 12/100
23/23 - 0s - loss: 185.0543 - val_lo

Epoch 97/100
23/23 - 0s - loss: 34.1979 - val_loss: 145.0511 - 86ms/epoch - 4ms/step
Epoch 98/100
23/23 - 0s - loss: 33.6820 - val_loss: 144.9509 - 111ms/epoch - 5ms/step
Epoch 99/100
23/23 - 0s - loss: 33.7236 - val_loss: 143.1929 - 96ms/epoch - 4ms/step
Epoch 100/100
23/23 - 0s - loss: 33.5076 - val_loss: 156.7142 - 81ms/epoch - 4ms/step


<h2> NOTE.</h2>

Feel free to vary the following and note what impact each change has on the model's performance:

1. Increase or decreate number of neurons in hidden layers
2. Add more hidden layers
3. Increase number of epochs